# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [80]:
csv_path = 'outputdata/cities_summary.csv'
cities_df = pd.read_csv(csv_path)
cities_df.head()

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,kahului,20.8947,-156.4700,82.98,74,40,20.71,US,1627327554
1,port alfred,-33.5906,26.8910,64.51,35,100,10.02,ZA,1627327571
2,narrabri,-30.3167,149.7833,35.69,87,0,0.00,AU,1627327572
3,broome,42.2506,-75.8330,88.02,47,20,1.99,US,1627327573
4,panlaitan,12.1191,119.8486,83.34,82,92,20.85,PH,1627327397


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [81]:
# Configure gmaps with personal API key
gmaps.configure(api_key = g_key)

In [94]:
# locations on df
all_locations = cities_df[["Lat", "Lon"]]

# Humidity
humidity = cities_df["Humidity"]

In [83]:
# Map + Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [84]:
cities_df.head()

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,kahului,20.8947,-156.4700,82.98,74,40,20.71,US,1627327554
1,port alfred,-33.5906,26.8910,64.51,35,100,10.02,ZA,1627327571
2,narrabri,-30.3167,149.7833,35.69,87,0,0.00,AU,1627327572
3,broome,42.2506,-75.8330,88.02,47,20,1.99,US,1627327573
4,panlaitan,12.1191,119.8486,83.34,82,92,20.85,PH,1627327397


In [85]:
# Ideal weather conditions for a nice sunny vacation

# Temperature conditions - between 70 and 80 degrees F
temp_con = (cities_df["Max Temp (F)"] >= 70) & (cities_df["Max Temp (F)"] <= 80)

# Humidity conditions - between 60 to 75 precent
humidity_con = (cities_df["Humidity"] >= 60) & (cities_df["Humidity"] <= 75)

# Cloudiness conditions - below 10 percent
cloud_con = (cities_df["Cloudiness"] < 10)

# Conditions df
conditions_df = cities_df.loc[temp_con & humidity_con & cloud_con]
conditions_df

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
7,caravelas,-17.7125,-39.2481,75.04,69,5,7.74,BR,1627327576
29,nador,35.1740,-2.9287,78.87,73,0,6.91,MA,1627327363
53,arraial do cabo,-22.9661,-42.0278,78.76,64,0,18.41,BR,1627327611
64,atuona,-9.8000,-139.0333,78.04,68,4,25.10,PF,1627327359
135,bengkulu,-3.8004,102.2655,74.21,75,0,4.36,ID,1627327653
146,bandarbeyla,9.4942,50.8122,78.01,75,2,27.45,SO,1627327657
248,masallatah,32.6167,14.0000,76.87,69,0,16.53,LY,1627327697
258,tukrah,32.5341,20.5791,78.49,62,0,18.23,LY,1627327701
356,paita,-5.0892,-81.1144,79.36,70,1,14.61,PE,1627327739
420,sao joao da barra,-21.6403,-41.0511,78.89,74,0,18.39,BR,1627327764


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
# Assign new df and add hotel column
hotel_df = conditions_df
hotel_df['Hotel Name'] = ""

hotel_df.head()

<ipython-input-86-ea4b6e38a082>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,caravelas,-17.7125,-39.2481,75.04,69,5,7.74,BR,1627327576,
29,nador,35.1740,-2.9287,78.87,73,0,6.91,MA,1627327363,
53,arraial do cabo,-22.9661,-42.0278,78.76,64,0,18.41,BR,1627327611,
64,atuona,-9.8000,-139.0333,78.04,68,4,25.10,PF,1627327359,
135,bengkulu,-3.8004,102.2655,74.21,75,0,4.36,ID,1627327653,


In [87]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iteration through hotel_df:
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lon = row["Lon"]
    city = row["City"]
    target_search = "Hotel"
    target_radius = 5000
    target_type = "Hotel"
    target_rank = "distance"
    
    # parameteres
    params={
        "location": f"{lat}, {lon}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        # "rankby": target_rank,
        "key": g_key,
    }
    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params = params).json()
    
    # extract results:
    results = response ['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing data... skipping.")
        
    print("------------")
    
print("Search complete.")

Retrieving Results for Index 7: caravelas.
Closest hotel is Pousada Encanto Abrolhos.
------------
Retrieving Results for Index 29: nador.


/home/mrmrrpg/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is Hotel Marchica Lagoon Resort, Nador Morocco.
------------
Retrieving Results for Index 53: arraial do cabo.
Closest hotel is Orlanova Boutique Hotel.
------------
Retrieving Results for Index 64: atuona.
Closest hotel is Hôtel Hiva Oa Hanakee Lodge.
------------
Retrieving Results for Index 135: bengkulu.
Closest hotel is Mercure Bengkulu.
------------
Retrieving Results for Index 146: bandarbeyla.
Closest hotel is JABIR HOTEL.
------------
Retrieving Results for Index 248: masallatah.
Missing data... skipping.
------------
Retrieving Results for Index 258: tukrah.
Closest hotel is مصيف مرحبا للعائلات.
------------
Retrieving Results for Index 356: paita.
Closest hotel is Hotel Náutico de Paita.
------------
Retrieving Results for Index 420: sao joao da barra.
Closest hotel is Pousada Kactus.
------------
Retrieving Results for Index 433: sfantu gheorghe.
Closest hotel is Hotel Sugás.
------------
Retrieving Results for Index 448: vila franca do campo.
Closest hotel is

In [88]:
hotel_df

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,caravelas,-17.7125,-39.2481,75.04,69,5,7.74,BR,1627327576,Pousada Encanto Abrolhos
29,nador,35.1740,-2.9287,78.87,73,0,6.91,MA,1627327363,"Hotel Marchica Lagoon Resort, Nador Morocco"
53,arraial do cabo,-22.9661,-42.0278,78.76,64,0,18.41,BR,1627327611,Orlanova Boutique Hotel
64,atuona,-9.8000,-139.0333,78.04,68,4,25.10,PF,1627327359,Hôtel Hiva Oa Hanakee Lodge
135,bengkulu,-3.8004,102.2655,74.21,75,0,4.36,ID,1627327653,Mercure Bengkulu
146,bandarbeyla,9.4942,50.8122,78.01,75,2,27.45,SO,1627327657,JABIR HOTEL
248,masallatah,32.6167,14.0000,76.87,69,0,16.53,LY,1627327697,
258,tukrah,32.5341,20.5791,78.49,62,0,18.23,LY,1627327701,مصيف مرحبا للعائلات
356,paita,-5.0892,-81.1144,79.36,70,1,14.61,PE,1627327739,Hotel Náutico de Paita
420,sao joao da barra,-21.6403,-41.0511,78.89,74,0,18.39,BR,1627327764,Pousada Kactus


In [104]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [105]:
marker_locations = hotel_df[["Lat", "Lon"]]
city = hotel_df["City"]
hotel = hotel_df["Hotel Name"]
country = hotel_df["Country"]

# Map + Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(all_locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                point_radius = 1)

markers = gmaps.marker_layer(marker_locations,
            info_box_content= info_box_template
)

fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))